In [ ]:
using ContGridMod
using Plots

In [ ]:
# TODO make an example/explanation on the boundary conditions
data, ~ = import_json_numerics("../data/test/star10.json")
#data, ~ = import_json_numerics("../data/test/triangle.json")
border = data["border"];

In [ ]:
dx = 0.02
mesh = get_mesh(border, dx);

In [ ]:
# plot inside and border
scatter(mesh.coord[:, 1], mesh.coord[:, 2], label="grid nodes")
plot!(border[:, 1], border[:, 2], label="border line", color=:black, linewidth=2.0)
plot!(aspect_ratio=:equal)